In [209]:
import pandas as pd
import numpy as np

In [210]:
data = pd.Series(np.random.randn(9),
                index=[["a", "a", "a", "b", "b", "c", "c", "d", "d"], 
                       [1, 2, 3, 1, 3, 1, 2, 2, 3]])

data

a  1   -1.128860
   2   -1.127005
   3    0.183443
b  1   -0.621832
   3    0.333963
c  1    1.327556
   2    2.807051
d  2   -1.090975
   3    2.643444
dtype: float64

In [211]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [212]:
data.loc[:, 2]

a   -1.127005
c    2.807051
d   -1.090975
dtype: float64

In [213]:
data.unstack()

,1,2,3
a,-1.128860,-1.127005,0.183443
b,-0.621832,NaN,0.333963
c,1.327556,2.807051,NaN
d,NaN,-1.090975,2.643444


In [214]:
# inverse of stack
data.unstack().stack()

a  1   -1.128860
   2   -1.127005
   3    0.183443
b  1   -0.621832
   3    0.333963
c  1    1.327556
   2    2.807051
d  2   -1.090975
   3    2.643444
dtype: float64

## Multindex on Rows and Cols

In [215]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])

frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [216]:
frame.index.names = ["key1", "key2"]
frame.columns.names = ["State", "Color"]

frame

State      Ohio     Colorado
Color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [217]:
frame.swaplevel("key1", "key2")

State      Ohio     Colorado
Color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [218]:
frame.sort_index(level=1)

State      Ohio     Colorado
Color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [219]:
frame.swaplevel(0, 1).sort_index(level=0)

State      Ohio     Colorado
Color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics by Level

Note that the book points out that 
```python
frame.sum(level=1)
```
But it's deprecated, so use
```python
frame.groupby(level=1).sum()
```

In [220]:
frame.groupby(level='key2').sum()

State  Ohio     Colorado
Color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [221]:
frame.groupby(level="Color", axis=1).sum()

Color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## Indexing with DataFrame column

In [222]:
frame = pd.DataFrame(
    {
        'a': range(7),
        'b': range(7, 0, -1),
        'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
        'd': [0, 1, 2, 0, 1, 2, 3]
    }
)

frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [223]:
# creating new DF using one or more of its col as index
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [224]:
# we could make the column stay 
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [225]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# Combining and Merging Datasets

## Databese-Style DataFrame joins

In [226]:
df1 = pd.DataFrame({ 'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data': range(7) })

df2 = pd.DataFrame({ 'key': ['a', 'b', 'd'],
                    'data': range(3) })

In [227]:
df1

,key,data
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [228]:
df2

,key,data
0,a,0
1,b,1
2,d,2


In [229]:
pd.merge(df1, df2, on='key')

,key,data_x,data_y
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [230]:
pd.merge(df1, df2, on='key', how="outer", indicator=True)

,key,data_x,data_y,_merge
0,b,0.0,1.0,both
1,b,1.0,1.0,both
2,b,6.0,1.0,both
3,a,2.0,0.0,both
4,a,4.0,0.0,both
5,a,5.0,0.0,both
6,c,3.0,NaN,left_only
7,d,NaN,2.0,right_only


It's possible to specify:
- `inner`
- `left`
- `right`
- `outer`: Use all key combinations in both tables together.

## Mergin on index

In [231]:
left1 = pd.DataFrame({
    'key': ['a', 'b', 'a', 'a', 'b', 'c'],
    'value': range(6)
})

right1 = pd.DataFrame({
    'group_val': [3.5, 7]
}, index=['a', 'b'])

In [232]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [233]:
right1

,group_val
a,3.5
b,7.0


In [234]:
pd.merge(left1, right1, left_on="key", right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [235]:
pd.merge(left1, right1, left_on="key", right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


### Hierarchly Index data

In [236]:
lefth = pd.DataFrame({
    'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    'key2': [2000, 2001, 2002, 2001, 2002],
    'data': np.arange(5.0)
})

righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [237]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [238]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [239]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


#### `join` clause to join only on indexes

In [240]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'b', 'e'],
                     columns=['Ohio', 'Nevada'])

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13., 14.]],
                     index=['b', 'c', 'd', 'e'],
                     columns=['Missouri', 'Alabama'])

In [241]:
left2

,Ohio,Nevada
a,1.0,2.0
b,3.0,4.0
e,5.0,6.0


In [242]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [243]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,3.0,4.0,7.0,8.0
c,NaN,NaN,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [244]:
# The same could be archived with:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,3.0,4.0,7.0,8.0
c,NaN,NaN,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [245]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])

In [246]:
# multiple jonis
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,3.0,4.0,7.0,8.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0


## Concatenating Along an Axis

In [247]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

In [248]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [249]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [250]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


### Combining Data with Overlap

In [251]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])

b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])


In [252]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [253]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [254]:
# sort of if-else expression
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, 5. ])

In [255]:
# same as before but with Series
b[:-2].combine_first(a[2:]).sort_index(ascending=False)

f    0.0
e    1.0
d    2.0
c    3.0
b    4.5
a    NaN
dtype: float64

In [256]:
df1 = pd.DataFrame({
    'a': [1., np.nan, 5., np.nan],
    'b': [np.nan, 2., np.nan, 6.],
    'c': range(2, 18, 4)
})

df2 = pd.DataFrame({
    'a': [5., 4., np.nan, 3., 7.],
    'b': [np.nan, 3., 4., 6., 8.]
})

In [257]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [258]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [259]:

df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# Reshaping and Pivoting

In [260]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'], name='number'));

data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [261]:
# Pivoting the columns into the rows, produicing a pd.Series
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [262]:
# From a hierarchically indexed Series, it's possiible to make a dataframe
# By deafult, the innermost index is unstacked into columns
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [263]:
# You can specify which level of the indexes to unstack
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [264]:
# You can accomplish the same thing with index name
result.unstack("state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


#### Handling missing values when unstacking

In [265]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

data = pd.concat([s1, s2], keys=['one', 'two'])
data

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [266]:
data.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [267]:
data.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [268]:
df = pd.DataFrame({
    'left': result,
    'right': result + 5},
    columns=pd.Index(['left', 'right'], name='side'))

df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [269]:
df.stack()

state     number  side 
Ohio      one     left      0
                  right     5
          two     left      1
                  right     6
          three   left      2
                  right     7
Colorado  one     left      3
                  right     8
          two     left      4
                  right     9
          three   left      5
                  right    10
dtype: int64

In [270]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [271]:
df.unstack('number')

side     left           right          
number    one two three   one two three
state                                  
Ohio        0   1     2     5   6     7
Colorado    3   4     5     8   9    10

In [272]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting Long to Wide Format

In [273]:
data = pd.read_csv("../../datasets/macrodata.csv")

In [274]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [275]:
periods = pd.PeriodIndex(year=data['year'], quarter=data['quarter'], name="date")

In [276]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2
...,...,...,...
198,13324.600,-3.16,6.0
199,13141.920,-8.79,6.9
200,12925.410,0.94,8.1
201,12901.504,3.37,9.2


In [277]:
data.index = periods.to_timestamp('D', 'end')
data

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,13324.600,-3.16,6.0
2008-12-31 23:59:59.999999999,13141.920,-8.79,6.9
2009-03-31 23:59:59.999999999,12925.410,0.94,8.1


In [278]:
# Stacking puts every column (realdgp, infl, enemp) to be
# stored as an index in the second level, and the 
# values stays in their same rows.

# In the other hand, reset_index() pops the indexes 
# (the one called date and item) and make them as a row

# Now the value column has been replaced with the name '0',
# so a simple rename sould do the trick
ldata = data.stack().reset_index().rename(columns={0: 'value'})
ldata

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
...,...,...,...
604,2009-06-30 23:59:59.999999999,infl,3.370
605,2009-06-30 23:59:59.999999999,unemp,9.200
606,2009-09-30 23:59:59.999999999,realgdp,12990.341
607,2009-09-30 23:59:59.999999999,infl,3.560


In [279]:
pivoted = ldata.pivot(index='date', columns='item', values='value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [280]:
ldata['value2'] = np.random.randn(len(ldata))
ldata.pivot(index='date', columns='item')

value                     value2            \
item                           infl    realgdp unemp      infl   realgdp   
date                                                                       
1959-03-31 23:59:59.999999999  0.00   2710.349   5.8  0.195010  0.865886   
1959-06-30 23:59:59.999999999  2.34   2778.801   5.1  1.637641 -0.096724   
1959-09-30 23:59:59.999999999  2.74   2775.488   5.3  1.146101  0.325297   
1959-12-31 23:59:59.999999999  0.27   2785.204   5.6  0.218326 -1.023029   
1960-03-31 23:59:59.999999999  2.31   2847.699   5.2 -0.111649 -1.939057   
...                             ...        ...   ...       ...       ...   
2008-09-30 23:59:59.999999999 -3.16  13324.600   6.0 -0.966704  0.715059   
2008-12-31 23:59:59.999999999 -8.79  13141.920   6.9  1.573577  0.623817   
2009-03-31 23:59:59.999999999  0.94  12925.410   8.1  0.459821  0.627869   
2009-06-30 23:59:59.999999999  3.37  12901.504   9.2 -1.327537 -1.169242   
2009-09-30 23:59:59.999999999  3.56  12990.341   9.6  0.642506  0.111452   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999 -0.875703  
1959-06-30 23:59:59.999999999  0.814043  
1959-09-30 23:59:59.999999999 -1.185506  
1959-12-31 23:59:59.999999999  0.781296  
1960-03-31 23:59:59.999999999  0.502942  
...                                 ...  
2008-09-30 23:59:59.999999999  0.402345  
2008-12-31 23:59:59.999999999  0.079060  
2009-03-31 23:59:59.999999999  1.413015  
2009-06-30 23:59:59.999999999  0.445022  
2009-09-30 23:59:59.999999999  1.025436  

[203 rows x 6 columns]

### Pivoting from Wide to Long format

In [281]:
df = pd.DataFrame({
    'key': ['foo', 'bar', 'baz'],
    'A': [1, 2, 3],
    'B': [4, 5, 6],
    'C': [7, 8, 9]
})

df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [282]:
# Unpivoting table, using 'key' as identifier variaible
meleted = pd.melt(df, id_vars='key')
meleted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [283]:
# It's possibile to reshape back thanks to pivot
meleted.pivot(index='key', columns='variable', values='value').reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7
